# In this notebook we will train the LSTM model

In [ ]:
drop_seq_4h = [
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'bollinger_upper', 'bollinger_lower', 'MACD_line', 'MACD_signal', 'stoch_%D',
    'EMA_21', 'SMA_20',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'above_sma20', 'above_sma50', 'ema7_above_ema21',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'macd_positive', 'momentum_alignment', 'macd_rising', 'obv_rising_24h',
    'trending_market', 'trend_alignment',
    'support_level', 'resistance_level', 'volatility_regime',
    'close_daily', 'rsi_daily'
]

✅ DROP_COLS['4H']['LSTM']
✅ DROP_COLS['4H']['GRU']
✅ DROP_COLS['4H']['CNN']
✅ DROP_COLS['4H']['CNN_LSTM']
✅ DROP_COLS['4H']['TCN']


In [ ]:
drop columns catnboost
drop_catboost_4h = [
    'open', 'high', 'low',
    'high_low', 'high_close', 'low_close', 'typical_price',
    'EMA_21', 'SMA_20', 'bollinger_upper', 'bollinger_lower',
    'MACD_line', 'MACD_signal', 'stoch_%D',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3', 
    'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'rsi_oversold', 'rsi_overbought', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'momentum_alignment',
    'ema7_above_ema21', 'obv_rising_24h', 'above_sma20', 'above_sma50',
    'macd_positive', 'macd_rising'
]


In [ ]:
# drop columns of xgboost + LightGBM are the same

In [ ]:
# =============================================================
#  OPTIMIZED CUDA-ENABLED LSTM/GRU SUCCESSIVE-HALVING TUNER
# =============================================================
import warnings, json, gc, time, os
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd

# ─── Silence warnings ────────────────────────────────────────
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

# ─── TensorFlow imports ───────────────────────────────────────
import tensorflow as tf
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras import mixed_precision

# ─── Sklearn imports ──────────────────────────────────────────
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# ─── Reproducibility ──────────────────────────────────────────
np.random.seed(42)
tf.random.set_seed(42)

# =============================================================
# IMPROVED GPU SETUP
# =============================================================
def setup_gpu():
    """Properly configure GPU with better error handling"""
    print("🔧 Setting up GPU...")
    
    # List available devices
    gpus = tf.config.list_physical_devices('GPU')
    if not gpus:
        print("⚠️  No GPU detected - running on CPU (will be slower)")
        return False
    
    try:
        # Configure GPU memory growth for all GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Set mixed precision policy (but make it optional)
        try:
            mixed_precision.set_global_policy('mixed_float16')
            print(f"✅ {len(gpus)} GPU(s) configured with mixed_float16")
        except Exception as e:
            print(f"⚠️  Mixed precision failed ({e}), using float32")
            mixed_precision.set_global_policy('float32')
        
        # Test GPU availability
        with tf.device('/GPU:0'):
            test_tensor = tf.constant([1.0, 2.0, 3.0])
            result = tf.reduce_sum(test_tensor)
        
        print(f"✅ GPU test successful: {result.numpy()}")
        return True
        
    except Exception as e:
        print(f"❌ GPU setup failed: {e}")
        print("Falling back to CPU...")
        return False

# =============================================================
# CONFIGURATION
# =============================================================
CONFIG = {
    'data': {
        'csv_file': Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv"),
        'time_col': 'timestamp',
        'target_col': 'target',
        'start_date': '2018-01-01',
        'test_frac': 0.20,
        'val_frac': 0.15,
        'drop_cols': [
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'bollinger_upper', 'bollinger_lower', 'MACD_line', 'MACD_signal', 'stoch_%D',
    'EMA_21', 'SMA_20',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'above_sma20', 'above_sma50', 'ema7_above_ema21',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'macd_positive', 'momentum_alignment', 'macd_rising', 'obv_rising_24h',
    'trending_market', 'trend_alignment',
    'support_level', 'resistance_level', 'volatility_regime',
    'close_daily', 'rsi_daily'
],
        'bounded_cols': ['rsi', 'stoch_%K', 'bb_position', 'williams_%R']
    }
}

# =============================================================
# OPTIMIZED DATA PROCESSING
# =============================================================
class OptimizedDataProcessor:
    """Streamlined data processing for better performance"""
    
    def __init__(self, config):
        self.config = config['data']
        self.scalers = {}
        
    def load_and_prepare_data(self):
        """Load and prepare data efficiently"""
        print("📊 Loading data...")
        
        if not self.config['csv_file'].exists():
            raise FileNotFoundError(f"Data file not found: {self.config['csv_file']}")
        
        # Load data
        df = pd.read_csv(self.config['csv_file'], parse_dates=[self.config['time_col']])
        df = df.set_index(self.config['time_col']).sort_index()
        df = df.loc[self.config['start_date']:]
        
        # Prepare features and target
        cols_to_drop = list(set(self.config['drop_cols']) & set(df.columns)) + [self.config['target_col']]
        X = df.drop(columns=cols_to_drop)
        y = df[self.config['target_col']]
        
        # Handle missing values
        if X.isna().any().any():
            X = X.fillna(method='ffill').fillna(method='bfill')
        
        print(f"✅ Data loaded: {X.shape[0]} samples, {X.shape[1]} features")
        return X, y
    
    def create_sequences(self, X, y, seq_len):
        """Create sequences efficiently using numpy operations"""
        n_samples = len(X) - seq_len + 1
        
        # Pre-allocate arrays
        X_seq = np.empty((n_samples, seq_len, X.shape[1]), dtype=np.float32)
        y_seq = np.empty(n_samples, dtype=np.float32)
        
        # Use array views for efficiency
        X_values = X.values.astype(np.float32)
        y_values = y.values.astype(np.float32)
        
        for i in range(n_samples):
            X_seq[i] = X_values[i:i+seq_len]
            y_seq[i] = y_values[i+seq_len-1]
            
        return X_seq, y_seq
    
    def split_and_scale_data(self, X, y, seq_len, scaler_type='mixed'):
        """Split and scale data efficiently"""
        # Calculate split indices
        n_samples = len(X)
        test_idx = int(n_samples * (1 - self.config['test_frac']))
        val_idx = int(test_idx * (1 - self.config['val_frac']))
        
        # Split data
        X_train, X_val, X_test = X.iloc[:val_idx], X.iloc[val_idx:test_idx], X.iloc[test_idx:]
        y_train, y_val, y_test = y.iloc[:val_idx], y.iloc[val_idx:test_idx], y.iloc[test_idx:]
        
        # Scale features
        if scaler_type == 'mixed':
            # Use MinMax for bounded features, Standard for others
            bounded_mask = X_train.columns.isin(self.config['bounded_cols'])
            
            # Scale bounded features
            self.scalers['minmax'] = MinMaxScaler()
            X_train_bounded = self.scalers['minmax'].fit_transform(X_train.loc[:, bounded_mask])
            X_val_bounded = self.scalers['minmax'].transform(X_val.loc[:, bounded_mask])
            X_test_bounded = self.scalers['minmax'].transform(X_test.loc[:, bounded_mask])
            
            # Scale unbounded features
            self.scalers['standard'] = StandardScaler()
            X_train_unbounded = self.scalers['standard'].fit_transform(X_train.loc[:, ~bounded_mask])
            X_val_unbounded = self.scalers['standard'].transform(X_val.loc[:, ~bounded_mask])
            X_test_unbounded = self.scalers['standard'].transform(X_test.loc[:, ~bounded_mask])
            
            # Combine scaled features
            X_train_scaled = np.column_stack([X_train_bounded, X_train_unbounded]).astype(np.float32)
            X_val_scaled = np.column_stack([X_val_bounded, X_val_unbounded]).astype(np.float32)
            X_test_scaled = np.column_stack([X_test_bounded, X_test_unbounded]).astype(np.float32)
            
        else:
            # Use single scaler
            scaler_class = StandardScaler if scaler_type == 'standard' else MinMaxScaler
            self.scalers[scaler_type] = scaler_class()
            
            X_train_scaled = self.scalers[scaler_type].fit_transform(X_train).astype(np.float32)
            X_val_scaled = self.scalers[scaler_type].transform(X_val).astype(np.float32)
            X_test_scaled = self.scalers[scaler_type].transform(X_test).astype(np.float32)
        
        # Convert to DataFrames for sequence creation
        columns = X_train.columns
        X_train_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=columns)
        X_val_df = pd.DataFrame(X_val_scaled, index=X_val.index, columns=columns)
        X_test_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=columns)
        
        # Create sequences
        X_train_seq, y_train_seq = self.create_sequences(X_train_df, y_train, seq_len)
        X_val_seq, y_val_seq = self.create_sequences(X_val_df, y_val, seq_len)
        X_test_seq, y_test_seq = self.create_sequences(X_test_df, y_test, seq_len)
        
        return (X_train_seq, y_train_seq), (X_val_seq, y_val_seq), (X_test_seq, y_test_seq)

# =============================================================
# OPTIMIZED MODEL BUILDING
# =============================================================
class ModelBuilder:
    """Optimized model building with better GPU utilization"""
    
    @staticmethod
    def build_model(input_shape, config):
        """Build RNN model with optimizations"""
        model = Sequential()
        
        # Choose RNN layer
        rnn_layer = LSTM if config['layer_type'] == 'LSTM' else GRU
        
        # Ensure all numeric values are proper Python types
        units = int(config['units'])
        layers = int(config['layers'])
        dropout = float(config['dropout'])
        dense_units = int(config['dense_units'])
        l1_val = float(config.get('l1', 0.0))
        l2_val = float(config.get('l2', 0.0))
        learning_rate = float(config['learning_rate'])
        
        # Add RNN layers
        for i in range(layers):
            return_sequences = (i < layers - 1)
            
            if i == 0:
                model.add(rnn_layer(
                    units,
                    return_sequences=return_sequences,
                    input_shape=input_shape,
                    kernel_regularizer=l1_l2(l1_val, l2_val)
                ))
            else:
                model.add(rnn_layer(
                    units,
                    return_sequences=return_sequences,
                    kernel_regularizer=l1_l2(l1_val, l2_val)
                ))
        
        # Add dense layers
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        model.add(Dense(dense_units, activation=config.get('activation', 'relu')))
        model.add(Dropout(dropout))
        
        # Output layer with explicit dtype for mixed precision
        model.add(Dense(1, activation='sigmoid', dtype='float32'))
        
        # Compile model
        optimizer_map = {'adam': Adam, 'nadam': Nadam}
        optimizer = optimizer_map[config['optimizer']](learning_rate=learning_rate)
        
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

# =============================================================
# CALLBACKS AND METRICS
# =============================================================
class F1EarlyStopping(Callback):
    """Custom callback for F1-based early stopping"""
    
    def __init__(self, validation_data, threshold=0.5, patience=5, min_delta=0.001):
        super().__init__()
        self.X_val, self.y_val = validation_data
        self.threshold = threshold
        self.patience = patience
        self.min_delta = min_delta
        self.best_f1 = 0
        self.wait = 0
        
    def on_epoch_end(self, epoch, logs=None):
        # Get predictions
        y_pred_prob = self.model.predict(self.X_val, verbose=0).flatten()
        y_pred = (y_pred_prob >= self.threshold).astype(int)
        
        # Calculate F1 score
        current_f1 = f1_score(self.y_val, y_pred, zero_division=0)
        
        # Check for improvement
        if current_f1 > self.best_f1 + self.min_delta:
            self.best_f1 = current_f1
            self.wait = 0
        else:
            self.wait += 1
            
        if self.wait >= self.patience:
            self.model.stop_training = True

def calculate_metrics(model, X, y, threshold=0.5):
    """Calculate comprehensive metrics"""
    y_pred_prob = model.predict(X, verbose=0).flatten()
    y_pred = (y_pred_prob >= threshold).astype(int)
    
    try:
        auc = roc_auc_score(y, y_pred_prob)
    except ValueError:
        auc = 0.5  # Default for single-class cases
    
    return {
        'accuracy': accuracy_score(y, y_pred),
        'precision': precision_score(y, y_pred, zero_division=0),
        'recall': recall_score(y, y_pred, zero_division=0),
        'f1': f1_score(y, y_pred, zero_division=0),
        'auc': auc
    }

# =============================================================
# OPTIMIZED HYPERPARAMETER SEARCH
# =============================================================
class SuccessiveHalvingTuner:
    """Successive halving hyperparameter tuner optimized for GPU"""
    
    def __init__(self, search_space, data_processor):
        self.search_space = search_space
        self.data_processor = data_processor
        self.results = []
        
    def sample_config(self):
        """Sample a random configuration"""
        config = {}
        for param, values in self.search_space.items():
            if isinstance(values, (list, tuple)):
                sampled_value = np.random.choice(values)
                # Convert numpy types to native Python types
                if hasattr(sampled_value, 'item'):
                    config[param] = sampled_value.item()
                else:
                    config[param] = sampled_value
            elif isinstance(values, dict):
                if 'min' in values and 'max' in values:
                    if values.get('log', False):
                        config[param] = np.random.uniform(
                            np.log10(values['min']), 
                            np.log10(values['max'])
                        )
                        config[param] = float(10 ** config[param])
                    else:
                        config[param] = float(np.random.uniform(values['min'], values['max']))
        return config
    
    def run_search(self, X, y, n_initial=24, n_survivors=6, quick_epochs=8, full_epochs=30):
        """Run successive halving search"""
        print(f"\n🚀 Starting Successive Halving Search")
        print(f"Initial candidates: {n_initial}, Survivors: {n_survivors}")
        
        # Stage 1: Quick evaluation
        print(f"\n📊 Stage 1: Quick evaluation ({quick_epochs} epochs)")
        stage1_results = []
        
        for i in range(n_initial):
            config = self.sample_config()
            print(f"\n⚡ Config {i+1}/{n_initial}: {config}")
            
            try:
                # Prepare data
                train_data, val_data, test_data = self.data_processor.split_and_scale_data(
                    X, y, int(config['sequence_length']), config.get('scaler', 'mixed')
                )
                
                X_train, y_train = train_data
                X_val, y_val = val_data
                
                # Build and train model
                model = ModelBuilder.build_model((X_train.shape[1], X_train.shape[2]), config)
                
                # Create callbacks
                callbacks = [
                    F1EarlyStopping((X_val, y_val), float(config.get('threshold', 0.5)), patience=3),
                    EarlyStopping(patience=3, restore_best_weights=True, verbose=0)
                ]
                
                # Train model
                with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
                    history = model.fit(
                        X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=quick_epochs,
                        batch_size=int(config.get('batch_size', 32)),
                        verbose=0,
                        callbacks=callbacks
                    )
                
                # Calculate validation F1
                metrics = calculate_metrics(model, X_val, y_val, float(config.get('threshold', 0.5)))
                f1_score = metrics['f1']
                
                stage1_results.append((f1_score, config, metrics))
                print(f"   F1: {f1_score:.4f}")
                
                # Clean up
                del model
                tf.keras.backend.clear_session()
                gc.collect()
                
            except Exception as e:
                print(f"   ❌ Failed: {str(e)}")
                stage1_results.append((0.0, config, {}))
        
        # Select top performers
        stage1_results.sort(key=lambda x: x[0], reverse=True)
        top_configs = [result[1] for result in stage1_results[:n_survivors]]
        top_f1_scores = [result[0] for result in stage1_results[:n_survivors]]
        
        print(f"\n🔥 Top {n_survivors} configs from Stage 1:")
        for i, f1 in enumerate(top_f1_scores):
            print(f"   Rank {i+1}: F1 = {f1:.4f}")
        
        # Stage 2: Full training
        print(f"\n🚀 Stage 2: Full training ({full_epochs} epochs)")
        best_f1 = 0
        best_config = None
        best_test_metrics = None
        
        for i, config in enumerate(top_configs):
            print(f"\n🔥 Training config {i+1}/{n_survivors}: {config}")
            
            try:
                # Prepare data
                train_data, val_data, test_data = self.data_processor.split_and_scale_data(
                    X, y, int(config['sequence_length']), config.get('scaler', 'mixed')
                )
                
                X_train, y_train = train_data
                X_val, y_val = val_data
                X_test, y_test = test_data
                
                # Build and train model
                model = ModelBuilder.build_model((X_train.shape[1], X_train.shape[2]), config)
                
                # Create callbacks
                callbacks = [
                    F1EarlyStopping((X_val, y_val), float(config.get('threshold', 0.5)), patience=5),
                    EarlyStopping(patience=5, restore_best_weights=True, verbose=0)
                ]
                
                # Train model
                with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
                    history = model.fit(
                        X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=full_epochs,
                        batch_size=int(config.get('batch_size', 32)),
                        verbose=0,
                        callbacks=callbacks
                    )
                
                # Calculate validation and test metrics
                val_metrics = calculate_metrics(model, X_val, y_val, float(config.get('threshold', 0.5)))
                test_metrics = calculate_metrics(model, X_test, y_test, float(config.get('threshold', 0.5)))
                
                print(f"   Val F1: {val_metrics['f1']:.4f}, Test F1: {test_metrics['f1']:.4f}")
                
                # Update best if improved
                if val_metrics['f1'] > best_f1:
                    best_f1 = val_metrics['f1']
                    best_config = config
                    best_test_metrics = test_metrics
                
                # Clean up
                del model
                tf.keras.backend.clear_session()
                gc.collect()
                
            except Exception as e:
                print(f"   ❌ Failed: {str(e)}")
        
        return best_config, best_f1, best_test_metrics

# =============================================================
# SEARCH SPACE DEFINITION
# =============================================================
SEARCH_SPACE = {
    'sequence_length': [16, 20, 24, 32],
    'units': [64, 96, 128],
    'layers': [1, 2],
    'layer_type': ['LSTM', 'GRU'],
    'dropout': [0.2, 0.3, 0.4],
    'learning_rate': {'min': 1e-4, 'max': 1e-2, 'log': True},
    'batch_size': [32, 64],
    'optimizer': ['adam', 'nadam'],
    'scaler': ['mixed', 'standard'],
    'l1': [0.0, 1e-4, 1e-3],
    'l2': [0.0, 1e-4, 1e-3],
    'dense_units': [32, 48, 64],
    'activation': ['relu', 'tanh'],
    'threshold': [0.4, 0.5, 0.6]
}

# =============================================================
# MAIN EXECUTION
# =============================================================
# =============================================================
# MAIN EXECUTION
# =============================================================
def test_basic_functionality():
    """Test basic functionality before running full search"""
    print("🧪 Testing basic functionality...")
    
    # Test model building
    test_config = {
        'units': 64,
        'layers': 1,
        'layer_type': 'LSTM',
        'dropout': 0.2,
        'dense_units': 32,
        'l1': 0.0,
        'l2': 0.0,
        'activation': 'relu',
        'optimizer': 'adam',
        'learning_rate': 0.001
    }
    
    try:
        model = ModelBuilder.build_model((10, 5), test_config)
        print("✅ Model building test passed")
        
        # Test with dummy data
        X_dummy = np.random.random((32, 10, 5)).astype(np.float32)
        y_dummy = np.random.randint(0, 2, 32).astype(np.float32)
        
        model.fit(X_dummy, y_dummy, epochs=1, verbose=0)
        print("✅ Model training test passed")
        
        # Test predictions
        pred = model.predict(X_dummy[:5], verbose=0)
        print("✅ Model prediction test passed")
        
        del model
        tf.keras.backend.clear_session()
        return True
        
    except Exception as e:
        print(f"❌ Basic functionality test failed: {e}")
        return False

def main():
    """Main execution function"""
    start_time = time.time()
    
    # Setup GPU
    gpu_available = setup_gpu()
    
    # Test basic functionality first
    if not test_basic_functionality():
        print("❌ Basic tests failed. Please check your TensorFlow installation.")
        return
    
    # Initialize data processor
    data_processor = OptimizedDataProcessor(CONFIG)
    
    try:
        # Load data
        X, y = data_processor.load_and_prepare_data()
    except Exception as e:
        print(f"❌ Failed to load data: {e}")
        print("Please check your file path and data format.")
        return
    
    # Initialize tuner
    tuner = SuccessiveHalvingTuner(SEARCH_SPACE, data_processor)
    
    # Run search
    best_config, best_val_f1, test_metrics = tuner.run_search(X, y)
    
    # Print results
    print("\n" + "="*80)
    print("🏆 HYPERPARAMETER SEARCH COMPLETE")
    print("="*80)
    
    if best_config is not None:
        print(f"Best Validation F1: {best_val_f1:.4f}")
        print("\n📋 Best Configuration:")
        for key, value in best_config.items():
            print(f"   {key:<18}: {value}")
        
        print("\n📊 Test Set Performance:")
        for metric, value in test_metrics.items():
            print(f"   {metric:<12}: {value:.4f}")
    else:
        print("❌ No successful configurations found!")
        print("All configurations failed - please check your data and model setup.")
        return
    
    runtime = (time.time() - start_time) / 60
    print(f"\n⏱️  Total Runtime: {runtime:.1f} minutes")
    print(f"🔧 GPU Used: {'Yes' if gpu_available else 'No'}")
    
    # Save results only if we have a successful run
    if best_config is not None:
        results_dir = Path("results")
        results_dir.mkdir(exist_ok=True)
        
        results = {
            'best_config': best_config,
            'best_val_f1': float(best_val_f1),
            'test_metrics': {k: float(v) for k, v in test_metrics.items()},
            'runtime_minutes': runtime,
            'gpu_used': gpu_available,
            'timestamp': datetime.now().isoformat()
        }
        
        results_file = results_dir / "optimized_lstm_tuning_results.json"
        with open(results_file, 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"💾 Results saved to: {results_file}")
    else:
        print("❌ No results to save - all configurations failed.")
        
        # Debug information
        print("\n🔍 Debugging Information:")
        print("- Check that your CSV file path is correct")
        print("- Verify your data has the expected columns")
        print("- Ensure your target column contains binary values")
        print("- Try running with a simpler configuration first")

if __name__ == "__main__":
    main()

🔧 Setting up GPU...
⚠️  No GPU detected - running on CPU (will be slower)
🧪 Testing basic functionality...
✅ Model building test passed
✅ Model training test passed
✅ Model prediction test passed
📊 Loading data...
✅ Data loaded: 15855 samples, 43 features

🚀 Starting Successive Halving Search
Initial candidates: 24, Survivors: 6

📊 Stage 1: Quick evaluation (8 epochs)

⚡ Config 1/24: {'sequence_length': 24, 'units': 96, 'layers': 2, 'layer_type': 'LSTM', 'dropout': 0.4, 'learning_rate': 0.002892684180163678, 'batch_size': 64, 'optimizer': 'nadam', 'scaler': 'standard', 'l1': 0.0001, 'l2': 0.0001, 'dense_units': 32, 'activation': 'relu', 'threshold': 0.4}
   F1: 0.6523

⚡ Config 2/24: {'sequence_length': 24, 'units': 128, 'layers': 1, 'layer_type': 'LSTM', 'dropout': 0.3, 'learning_rate': 0.00015608202800330114, 'batch_size': 32, 'optimizer': 'adam', 'scaler': 'standard', 'l1': 0.0, 'l2': 0.001, 'dense_units': 48, 'activation': 'relu', 'threshold': 0.6}
   F1: 0.1952

⚡ Config 3/24: {'s

In [1]:
# =============================================================
#  FLAWLESS LSTM TRAINER WITH OPTIMAL PARAMETERS
# =============================================================
import warnings, os, time, json
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd

# ─── Silence warnings ────────────────────────────────────────
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")

# ─── TensorFlow imports ───────────────────────────────────────
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ─── Sklearn imports ──────────────────────────────────────────
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# ─── Reproducibility ──────────────────────────────────────────
np.random.seed(42)
tf.random.set_seed(42)

# =============================================================
# CONFIGURATION WITH YOUR OPTIMAL PARAMETERS
# =============================================================
DATA_CONFIG = {
    'csv_file': Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv"),
    'time_col': 'timestamp',
    'target_col': 'target',
    'start_date': '2018-01-01',
    'test_frac': 0.20,
    'val_frac': 0.15,
    'drop_cols': [
        'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
        'bollinger_upper', 'bollinger_lower', 'MACD_line', 'MACD_signal', 'stoch_%D',
        'EMA_21', 'SMA_20',
        'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
        'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
        'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
        'bearish_scenario_4', 'bearish_scenario_6',
        'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
        'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
        'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
        'near_lower_band', 'overbought_reversal', 'oversold_reversal',
        'above_sma20', 'above_sma50', 'ema7_above_ema21',
        'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
        'macd_positive', 'momentum_alignment', 'macd_rising', 'obv_rising_24h',
        'trending_market', 'trend_alignment',
        'support_level', 'resistance_level', 'volatility_regime',
        'close_daily', 'rsi_daily'
    ]
}

# Your optimal hyperparameters
OPTIMAL_CONFIG = {
    'sequence_length': 24,
    'units': 96,
    'layers': 2,
    'layer_type': 'LSTM',
    'dropout': 0.4,
    'learning_rate': 0.002892684180163678,
    'batch_size': 64,
    'optimizer': 'nadam',
    'scaler': 'standard',
    'l1': 0.0001,
    'l2': 0.0001,
    'dense_units': 32,
    'activation': 'relu',
    'threshold': 0.4
}

# =============================================================
# DATA PROCESSING CLASS (SAME AS YOUR SEARCH)
# =============================================================
class OptimizedDataProcessor:
    """Streamlined data processing matching your hyperparameter search"""
    
    def __init__(self, data_config):
        self.config = data_config
        self.scaler = None
        
    def load_and_prepare_data(self):
        """Load and prepare data efficiently"""
        print("📊 Loading data...")
        
        if not self.config['csv_file'].exists():
            raise FileNotFoundError(f"Data file not found: {self.config['csv_file']}")
        
        # Load data
        df = pd.read_csv(self.config['csv_file'], parse_dates=[self.config['time_col']])
        df = df.set_index(self.config['time_col']).sort_index()
        df = df.loc[self.config['start_date']:]
        
        # Keep original timestamps for predictions
        original_timestamps = df.index.copy()
        
        # Prepare features and target
        cols_to_drop = list(set(self.config['drop_cols']) & set(df.columns)) + [self.config['target_col']]
        X = df.drop(columns=cols_to_drop)
        y = df[self.config['target_col']]
        
        # Handle missing values
        if X.isna().any().any():
            X = X.fillna(method='ffill').fillna(method='bfill')
        
        print(f"✅ Data loaded: {X.shape[0]} samples, {X.shape[1]} features")
        print(f"   Target distribution: {y.mean():.1%} positive, {1-y.mean():.1%} negative")
        
        return X, y, original_timestamps
    
    def create_sequences(self, X, y, seq_len):
        """Create sequences efficiently using numpy operations"""
        n_samples = len(X) - seq_len + 1
        
        # Pre-allocate arrays
        X_seq = np.empty((n_samples, seq_len, X.shape[1]), dtype=np.float32)
        y_seq = np.empty(n_samples, dtype=np.float32)
        
        # Use array views for efficiency
        X_values = X.values.astype(np.float32)
        y_values = y.values.astype(np.float32)
        
        for i in range(n_samples):
            X_seq[i] = X_values[i:i+seq_len]
            y_seq[i] = y_values[i+seq_len-1]
            
        return X_seq, y_seq
    
    def split_and_scale_data(self, X, y, timestamps, seq_len):
        """Split and scale data using standard scaler (matching your optimal config)"""
        # Calculate split indices
        n_samples = len(X)
        test_idx = int(n_samples * (1 - self.config['test_frac']))
        val_idx = int(test_idx * (1 - self.config['val_frac']))
        
        print(f"📊 Data splits:")
        print(f"   Train: samples 0 to {val_idx-1}")
        print(f"   Validation: samples {val_idx} to {test_idx-1}")
        print(f"   Test: samples {test_idx} to {n_samples-1}")
        
        # Split data
        X_train, X_val, X_test = X.iloc[:val_idx], X.iloc[val_idx:test_idx], X.iloc[test_idx:]
        y_train, y_val, y_test = y.iloc[:val_idx], y.iloc[val_idx:test_idx], y.iloc[test_idx:]
        ts_train = timestamps[:val_idx]
        ts_val = timestamps[val_idx:test_idx]
        ts_test = timestamps[test_idx:]
        
        # Scale features using StandardScaler (your optimal scaler)
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train).astype(np.float32)
        X_val_scaled = self.scaler.transform(X_val).astype(np.float32)
        X_test_scaled = self.scaler.transform(X_test).astype(np.float32)
        
        # Convert back to DataFrames for sequence creation
        columns = X_train.columns
        X_train_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=columns)
        X_val_df = pd.DataFrame(X_val_scaled, index=X_val.index, columns=columns)
        X_test_df = pd.DataFrame(X_test_scaled, index=X_test.index, columns=columns)
        
        # Create sequences
        X_train_seq, y_train_seq = self.create_sequences(X_train_df, y_train, seq_len)
        X_val_seq, y_val_seq = self.create_sequences(X_val_df, y_val, seq_len)
        X_test_seq, y_test_seq = self.create_sequences(X_test_df, y_test, seq_len)
        
        # Adjust timestamps for sequences
        ts_train_seq = ts_train[seq_len-1:]
        ts_val_seq = ts_val[seq_len-1:]
        ts_test_seq = ts_test[seq_len-1:]
        
        print(f"✅ Sequences created:")
        print(f"   Train: {X_train_seq.shape[0]} sequences")
        print(f"   Validation: {X_val_seq.shape[0]} sequences")
        print(f"   Test: {X_test_seq.shape[0]} sequences")
        
        return {
            'train': (X_train_seq, y_train_seq, ts_train_seq),
            'val': (X_val_seq, y_val_seq, ts_val_seq),
            'test': (X_test_seq, y_test_seq, ts_test_seq)
        }

# =============================================================
# MODEL BUILDING (MATCHING YOUR SEARCH ARCHITECTURE)
# =============================================================
def build_optimal_lstm_model(input_shape, config):
    """Build LSTM model with your optimal configuration"""
    print(f"🤖 Building optimal LSTM model...")
    print(f"   Architecture: {config['layers']} LSTM layers with {config['units']} units each")
    print(f"   Dense layer: {config['dense_units']} units ({config['activation']} activation)")
    print(f"   Dropout: {config['dropout']}")
    print(f"   L1/L2 regularization: {config['l1']}/{config['l2']}")
    
    model = Sequential()
    
    # Add LSTM layers
    for i in range(config['layers']):
        return_sequences = (i < config['layers'] - 1)
        
        if i == 0:
            model.add(LSTM(
                config['units'],
                return_sequences=return_sequences,
                input_shape=input_shape,
                kernel_regularizer=l1_l2(config['l1'], config['l2']),
                recurrent_regularizer=l1_l2(config['l1'], config['l2'])
            ))
        else:
            model.add(LSTM(
                config['units'],
                return_sequences=return_sequences,
                kernel_regularizer=l1_l2(config['l1'], config['l2']),
                recurrent_regularizer=l1_l2(config['l1'], config['l2'])
            ))
    
    # Add dense layers
    model.add(Dropout(config['dropout']))
    model.add(BatchNormalization())
    model.add(Dense(config['dense_units'], activation=config['activation']))
    model.add(Dropout(config['dropout']))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid', dtype='float32'))
    
    # Compile with your optimal optimizer and learning rate
    optimizer = Nadam(learning_rate=config['learning_rate'])
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"   Learning rate: {config['learning_rate']:.6f}")
    print(f"   Batch size: {config['batch_size']}")
    print(f"   Optimizer: {config['optimizer']}")
    
    return model

# =============================================================
# TRAINING AND EVALUATION
# =============================================================
def calculate_comprehensive_metrics(y_true, y_pred_prob, threshold=0.5):
    """Calculate all metrics"""
    y_pred = (y_pred_prob >= threshold).astype(int)
    
    try:
        auc = roc_auc_score(y_true, y_pred_prob)
    except ValueError:
        auc = 0.5
    
    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'auc': auc,
        'true_positives': int(tp),
        'false_positives': int(fp),
        'true_negatives': int(tn),
        'false_negatives': int(fn),
        'positive_predictions': int(y_pred.sum()),
        'total_predictions': len(y_pred),
        'positive_rate': float(y_pred.mean()),
        'threshold': threshold
    }

def train_optimal_lstm(data_splits, config, epochs=50):
    """Train LSTM with optimal parameters and proper callbacks"""
    
    # Extract data
    X_train, y_train, ts_train = data_splits['train']
    X_val, y_val, ts_val = data_splits['val']
    X_test, y_test, ts_test = data_splits['test']
    
    # Build model
    model = build_optimal_lstm_model((X_train.shape[1], X_train.shape[2]), config)
    
    # Setup improved callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=12,
            restore_best_weights=True,
            verbose=1,
            min_delta=0.0001
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        ModelCheckpoint(
            'best_model.h5',
            monitor='val_loss',
            save_best_only=True,
            verbose=0
        )
    ]
    
    # Train model
    print(f"\n🚀 Training optimal LSTM model...")
    print(f"   Max epochs: {epochs}")
    print(f"   Early stopping patience: 12")
    
    start_time = time.time()
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=config['batch_size'],
        callbacks=callbacks,
        verbose=1
    )
    
    training_time = time.time() - start_time
    print(f"✅ Training completed in {training_time/60:.1f} minutes")
    
    # Generate predictions and evaluate
    print(f"\n📊 Evaluating model performance...")
    
    results = {}
    predictions = {}
    
    for split_name, (X_split, y_split, ts_split) in data_splits.items():
        print(f"   Evaluating {split_name} set...")
        
        # Generate predictions
        y_pred_prob = model.predict(X_split, verbose=0).flatten()
        
        # Calculate metrics with optimal threshold
        metrics = calculate_comprehensive_metrics(y_split, y_pred_prob, config['threshold'])
        
        # Also calculate with 0.5 threshold for comparison
        metrics_05 = calculate_comprehensive_metrics(y_split, y_pred_prob, 0.5)
        
        results[split_name] = {
            'optimal_threshold': metrics,
            'threshold_05': metrics_05
        }
        
        predictions[split_name] = {
            'timestamps': ts_split,
            'y_true': y_split,
            'y_pred_prob': y_pred_prob,
            'y_pred_optimal': (y_pred_prob >= config['threshold']).astype(int),
            'y_pred_05': (y_pred_prob >= 0.5).astype(int)
        }
    
    return model, results, predictions, history, training_time

# =============================================================
# RESULTS DISPLAY AND EXPORT
# =============================================================
def print_detailed_results(results, config):
    """Print comprehensive results"""
    print(f"\n📊 DETAILED MODEL EVALUATION RESULTS")
    print("="*80)
    
    for split_name, split_results in results.items():
        print(f"\n🔍 {split_name.upper()} SET RESULTS:")
        
        # Results with optimal threshold
        opt_metrics = split_results['optimal_threshold']
        print(f"\n   With optimal threshold ({config['threshold']}):")
        print(f"     Accuracy:              {opt_metrics['accuracy']:.4f}")
        print(f"     Precision:             {opt_metrics['precision']:.4f}")
        print(f"     Recall:                {opt_metrics['recall']:.4f}")
        print(f"     F1-Score:              {opt_metrics['f1']:.4f}")
        print(f"     AUC:                   {opt_metrics['auc']:.4f}")
        print(f"     Positive predictions:  {opt_metrics['positive_predictions']}/{opt_metrics['total_predictions']} ({opt_metrics['positive_rate']:.1%})")
        
        # Confusion matrix
        print(f"     Confusion matrix:")
        print(f"       TP: {opt_metrics['true_positives']:4d} | FP: {opt_metrics['false_positives']:4d}")
        print(f"       FN: {opt_metrics['false_negatives']:4d} | TN: {opt_metrics['true_negatives']:4d}")
        
        # Comparison with 0.5 threshold
        comp_metrics = split_results['threshold_05']
        print(f"\n   Comparison with 0.5 threshold:")
        print(f"     Precision: {comp_metrics['precision']:.4f} vs {opt_metrics['precision']:.4f} (optimal)")
        print(f"     Recall:    {comp_metrics['recall']:.4f} vs {opt_metrics['recall']:.4f} (optimal)")
        print(f"     F1-Score:  {comp_metrics['f1']:.4f} vs {opt_metrics['f1']:.4f} (optimal)")

def create_predictions_csv(predictions, config, output_dir="results"):
    """Create separate CSV files for each split, with test as main output"""
    
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    saved_files = {}
    
    # Create CSV for each split
    for split_name, pred_data in predictions.items():
        split_df = pd.DataFrame({
            'timestamp': pred_data['timestamps'],
            'actual': pred_data['y_true'],
            'probability': pred_data['y_pred_prob'],
            'prediction_optimal': pred_data['y_pred_optimal'],
            'prediction_05': pred_data['y_pred_05'],
            'confidence': np.abs(pred_data['y_pred_prob'] - 0.5),
            'high_confidence': (np.abs(pred_data['y_pred_prob'] - 0.5) > 0.3).astype(int),
            'very_high_confidence': (np.abs(pred_data['y_pred_prob'] - 0.5) > 0.4).astype(int)
        })
        
        # Add useful columns
        split_df['correct_optimal'] = (split_df['actual'] == split_df['prediction_optimal']).astype(int)
        split_df['correct_05'] = (split_df['actual'] == split_df['prediction_05']).astype(int)
        split_df['date'] = split_df['timestamp'].dt.date
        split_df['year'] = split_df['timestamp'].dt.year
        split_df['month'] = split_df['timestamp'].dt.month
        split_df['hour'] = split_df['timestamp'].dt.hour
        
        # Save individual CSV
        csv_file = output_path / f"lstm_{split_name}_predictions_{timestamp}.csv"
        split_df.to_csv(csv_file, index=False)
        saved_files[split_name] = csv_file
        
        print(f"💾 {split_name.capitalize()} predictions saved: {csv_file} ({len(split_df)} rows)")
    
    # Create combined CSV as well (for completeness)
    all_data = []
    for split_name, pred_data in predictions.items():
        split_df = pd.DataFrame({
            'timestamp': pred_data['timestamps'],
            'split': split_name,
            'actual': pred_data['y_true'],
            'probability': pred_data['y_pred_prob'],
            'prediction_optimal': pred_data['y_pred_optimal'],
            'prediction_05': pred_data['y_pred_05'],
            'confidence': np.abs(pred_data['y_pred_prob'] - 0.5),
            'high_confidence': (np.abs(pred_data['y_pred_prob'] - 0.5) > 0.3).astype(int),
            'very_high_confidence': (np.abs(pred_data['y_pred_prob'] - 0.5) > 0.4).astype(int)
        })
        all_data.append(split_df)
    
    # Combined file
    combined_df = pd.concat(all_data, ignore_index=False)
    combined_df = combined_df.sort_values('timestamp').reset_index(drop=True)
    combined_df['correct_optimal'] = (combined_df['actual'] == combined_df['prediction_optimal']).astype(int)
    combined_df['correct_05'] = (combined_df['actual'] == combined_df['prediction_05']).astype(int)
    combined_df['date'] = combined_df['timestamp'].dt.date
    combined_df['year'] = combined_df['timestamp'].dt.year
    combined_df['month'] = combined_df['timestamp'].dt.month
    combined_df['hour'] = combined_df['timestamp'].dt.hour
    
    combined_file = output_path / f"lstm_all_predictions_{timestamp}.csv"
    combined_df.to_csv(combined_file, index=False)
    saved_files['combined'] = combined_file
    
    print(f"💾 Combined predictions saved: {combined_file} ({len(combined_df)} rows)")
    
    # Return test predictions as main DataFrame (most important)
    test_file = saved_files['test']
    test_df = pd.read_csv(test_file)
    
    print(f"\n🎯 MAIN OUTPUT: {test_file}")
    print(f"   Test set predictions: {len(test_df)} rows")
    
    return test_df, saved_files

def save_complete_results(model, results, predictions, config, training_time, output_dir="results"):
    """Save model and complete results"""
    
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save model
    model_file = output_path / f"optimal_lstm_model_{timestamp}.h5"
    model.save(model_file)
    print(f"💾 Model saved: {model_file}")
    
    # Save predictions CSVs (separate files for each split)
    test_predictions_df, saved_files = create_predictions_csv(predictions, config, output_dir)
    
    # Save complete results summary
    results_summary = {
        'optimal_config': config,
        'training_time_minutes': training_time / 60,
        'timestamp': datetime.now().isoformat(),
        'model_file': str(model_file),
        'prediction_files': {k: str(v) for k, v in saved_files.items()},
        'main_test_file': str(saved_files['test']),
        'data_config': DATA_CONFIG,
        'results_by_split': {
            split: {
                threshold_type: {k: float(v) if isinstance(v, (int, float, np.number)) else v 
                               for k, v in metrics.items()}
                for threshold_type, metrics in split_data.items()
            }
            for split, split_data in results.items()
        }
    }
    
    json_file = output_path / f"optimal_lstm_results_{timestamp}.json"
    with open(json_file, 'w') as f:
        json.dump(results_summary, f, indent=2, default=str)
    
    print(f"💾 Results summary saved: {json_file}")
    
    return test_predictions_df, saved_files, json_file

# =============================================================
# MAIN EXECUTION
# =============================================================
def main():
    """Main execution function"""
    print("🎯 FLAWLESS LSTM TRAINER WITH OPTIMAL PARAMETERS")
    print("="*60)
    print("Using your optimal configuration:")
    for key, value in OPTIMAL_CONFIG.items():
        print(f"   {key}: {value}")
    
    try:
        # Initialize data processor
        data_processor = OptimizedDataProcessor(DATA_CONFIG)
        
        # Load and prepare data
        X, y, timestamps = data_processor.load_and_prepare_data()
        
        # Split and scale data
        data_splits = data_processor.split_and_scale_data(
            X, y, timestamps, OPTIMAL_CONFIG['sequence_length']
        )
        
        # Train model
        model, results, predictions, history, training_time = train_optimal_lstm(
            data_splits, OPTIMAL_CONFIG, epochs=50
        )
        
        # Print detailed results
        print_detailed_results(results, OPTIMAL_CONFIG)
        
        # Save everything
        test_predictions_df, saved_files, json_file = save_complete_results(
            model, results, predictions, OPTIMAL_CONFIG, training_time
        )
        
        print(f"\n✅ TRAINING COMPLETE!")
        print(f"   🎯 MAIN TEST PREDICTIONS: {saved_files['test']}")
        print(f"   📁 All files saved:")
        for split_name, file_path in saved_files.items():
            print(f"      {split_name}: {file_path}")
        print(f"   🤖 Trained model: {saved_files.get('model', 'N/A')}")
        print(f"   📊 Results summary: {json_file}")
        
        # Final summary
        test_results = results['test']['optimal_threshold']
        print(f"\n🎯 FINAL TEST SET PERFORMANCE (threshold = {OPTIMAL_CONFIG['threshold']}):")
        print(f"   Precision: {test_results['precision']:.4f}")
        print(f"   Recall:    {test_results['recall']:.4f}")
        print(f"   F1-Score:  {test_results['f1']:.4f}")
        print(f"   AUC:       {test_results['auc']:.4f}")
        print(f"\n📄 Test predictions CSV: {len(test_predictions_df)} rows")
        
        return model, test_predictions_df, results
        
    except Exception as e:
        print(f"❌ Error during execution: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

if __name__ == "__main__":
    model, predictions_df, results = main()

🎯 FLAWLESS LSTM TRAINER WITH OPTIMAL PARAMETERS
Using your optimal configuration:
   sequence_length: 24
   units: 96
   layers: 2
   layer_type: LSTM
   dropout: 0.4
   learning_rate: 0.002892684180163678
   batch_size: 64
   optimizer: nadam
   scaler: standard
   l1: 0.0001
   l2: 0.0001
   dense_units: 32
   activation: relu
   threshold: 0.4
📊 Loading data...
✅ Data loaded: 15855 samples, 24 features
   Target distribution: 51.1% positive, 48.9% negative
📊 Data splits:
   Train: samples 0 to 10780
   Validation: samples 10781 to 12683
   Test: samples 12684 to 15854
✅ Sequences created:
   Train: 10758 sequences
   Validation: 1880 sequences
   Test: 3148 sequences
🤖 Building optimal LSTM model...
   Architecture: 2 LSTM layers with 96 units each
   Dense layer: 32 units (relu activation)
   Dropout: 0.4
   L1/L2 regularization: 0.0001/0.0001
   Learning rate: 0.002893
   Batch size: 64
   Optimizer: nadam

🚀 Training optimal LSTM model...
   Max epochs: 50
   Early stopping patie

169/169 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5108 - loss: 1.2528 - val_accuracy: 0.4894 - val_loss: 0.8672 - learning_rate: 0.0029
Epoch 2/50
165/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5049 - loss: 0.8414

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5049 - loss: 0.8406 - val_accuracy: 0.4840 - val_loss: 0.7712 - learning_rate: 0.0029
Epoch 3/50
165/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5061 - loss: 0.7616

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5061 - loss: 0.7613 - val_accuracy: 0.4840 - val_loss: 0.7363 - learning_rate: 0.0029
Epoch 4/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5079 - loss: 0.7324

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5079 - loss: 0.7324 - val_accuracy: 0.4840 - val_loss: 0.7174 - learning_rate: 0.0029
Epoch 5/50
165/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5141 - loss: 0.7151

...

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5289 - loss: 0.6944 - val_accuracy: 0.5441 - val_loss: 0.6917 - learning_rate: 7.2317e-04
Epoch 39/50
166/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5354 - loss: 0.6925

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5355 - loss: 0.6925 - val_accuracy: 0.5362 - val_loss: 0.6910 - learning_rate: 7.2317e-04
Epoch 40/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5321 - loss: 0.6928 - val_accuracy: 0.5383 - val_loss: 0.6928 - learning_rate: 7.2317e-04
Epoch 41/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5344 - loss: 0.6927 - val_accuracy: 0.5372 - val_loss: 0.6914 - learning_rate: 7.2317e-04
Epoch 42/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5402 - loss: 0.6920 - val_accuracy: 0.5399 - val_loss: 0.6917 - learning_rate: 7.2317e-04
Epoch 43/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5316 - loss: 0.6930 - val_accuracy: 0.5378 - val_loss: 0.6919 - learning_rate: 7.2317e-04
Epoch 44/50
165/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5278 - loss: 0.6953
Epoch 44: ReduceLROnPlateau reducing learning rate to 0.0003615855239331722.
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5299 - loss: 0.6934 - val_accuracy: 0.5394 - val_loss: 0.6903 - learning_rate: 3.6159e-04
Epoch 47/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5360 - loss: 0.6928 - val_accuracy: 0.5388 - val_loss: 0.6907 - learning_rate: 3.6159e-04
Epoch 48/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5354 - loss: 0.6915 - val_accuracy: 0.5383 - val_loss: 0.6906 - learning_rate: 3.6159e-04
Epoch 49/50
166/169 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5388 - loss: 0.6911

169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5388 - loss: 0.6911 - val_accuracy: 0.5346 - val_loss: 0.6898 - learning_rate: 3.6159e-04
Epoch 50/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5379 - loss: 0.6921 - val_accuracy: 0.5383 - val_loss: 0.6901 - learning_rate: 3.6159e-04
Restoring model weights from the end of the best epoch: 49.
✅ Training completed in 1.9 minutes

📊 Evaluating model performance...
   Evaluating train set...
   Evaluating val set...
   Evaluating test set...



📊 DETAILED MODEL EVALUATION RESULTS

🔍 TRAIN SET RESULTS:

   With optimal threshold (0.4):
     Accuracy:              0.5186
     Precision:             0.5155
     Recall:                0.9851
     F1-Score:              0.6769
     AUC:                   0.5626
     Positive predictions:  10521/10758 (97.8%)
     Confusion matrix:
       TP: 5424 | FP: 5097
       FN:   82 | TN:  155

   Comparison with 0.5 threshold:
     Precision: 0.5424 vs 0.5155 (optimal)
     Recall:    0.6502 vs 0.9851 (optimal)
     F1-Score:  0.5914 vs 0.6769 (optimal)

🔍 VAL SET RESULTS:

   With optimal threshold (0.4):
     Accuracy:              0.4920
     Precision:             0.4878
     Recall:                0.9857
     F1-Score:              0.6526
     AUC:                   0.5589
     Positive predictions:  1839/1880 (97.8%)
     Confusion matrix:
       TP:  897 | FP:  942
       FN:   13 | TN:   28

   Comparison with 0.5 threshold:
     Precision: 0.5156 vs 0.4878 (optimal)
     Recall: 